In [1]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv("data\\train.csv", index_col = False)
test = pd.read_csv("data\\test.csv", index_col = False)

In [3]:
test.weather.value_counts()

1    4221
2    1710
3     560
4       2
Name: weather, dtype: int64

In [4]:
train[:8]

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3


In [5]:
type(train)

pandas.core.frame.DataFrame

In [6]:
transformed_train = pd.core.frame.DataFrame(train)

In [7]:
import datetime
transformed_train['datetime'] = transformed_train['datetime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [8]:
transformed_train['year'] = transformed_train['datetime'].apply(lambda x:x.year)
transformed_train['month'] = transformed_train['datetime'].apply(lambda x:x.month)
transformed_train['day'] = transformed_train['datetime'].apply(lambda x:x.day)
transformed_train['hour'] = transformed_train['datetime'].apply(lambda x:x.hour)

In [9]:
transformed_train = transformed_train.drop(['datetime'], axis=1)

In [32]:
x_train.weather.value_counts()

1    4996
2    2024
3     599
4       1
Name: weather, dtype: int64

In [33]:
x_test = x_test.append(pd.DataFrame([[1,0,0,4,0,0,0,0,2011,1,1,1]], columns=list(x_test.columns)))

In [52]:
y_test = y_test.append(pd.DataFrame([[10]]))

In [31]:
from sklearn.model_selection import train_test_split
train_no_sale_price = transformed_train.drop(labels=['count','registered','casual'], axis=1)
train_sale_price = transformed_train['count']

x_train, x_test, y_train, y_test = train_test_split(train_no_sale_price, 
                                                    train_sale_price, 
                                                    test_size=0.3, 
                                                    random_state=4200)

In [34]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [ ]:
regressor.score(x_train,y_train)

In [ ]:
transformed_train.drop(['day'], axis=1)

In [35]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [ ]:
transformed_train.columns

In [36]:
categorical_columns = ['season', 'weather', 'year','month','hour','day']

In [37]:
train_categorical = pd.DataFrame({col: x_train[col] for col in categorical_columns})

In [ ]:
train_categorical

In [38]:
one_hot_encoder = enc.fit(train_categorical)

In [39]:
all_columns = list(transformed_train.columns)

In [ ]:
all_columns

In [40]:
non_categorical_columns = list(set(all_columns)-set(categorical_columns))

In [47]:
non_categorical_columns.remove('casual')

In [44]:
one_hot_x_train = one_hot_encoder.transform(train_categorical)

In [ ]:
non_categorical_columns

In [48]:
test_categorical = pd.DataFrame({col: x_test[col] for col in categorical_columns})
one_hot_encoder_test = OneHotEncoder().fit(test_categorical)
one_hot_x_test = one_hot_encoder_test.transform(test_categorical)
new_x_train = pd.np.concatenate([one_hot_x_train.todense(), x_train[non_categorical_columns]], axis=1)
new_x_test = pd.np.concatenate([one_hot_x_test.todense(), x_test[non_categorical_columns]], axis=1)
print(new_x_train.shape, new_x_test.shape)

(7620, 71) (3267, 71)


In [ ]:
all_data_lr = LinearRegression().fit(new_x_train, y_train)

In [49]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(new_x_train, y_train)

In [53]:
print(clf.score(new_x_train, y_train))
print(clf.score(new_x_test, y_test))

1.0
0.824108014907


In [108]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=1000,n_jobs=2, min_samples_split=2)
regr.fit(new_x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=2,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [109]:
print(regr.score(new_x_train, y_train))
print(regr.score(new_x_test, y_test))

0.988885863886
0.912682233748


In [56]:
import numpy as np

def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
    
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

In [95]:
real_values = np.squeeze(np.asarray(y_test.values))

In [96]:
predictions = regr.predict(new_x_test)

In [97]:
print(rmsle(predictions,real_values))

0.508774798773


In [ ]:
type(new_x_test)

In [ ]:
print(all_data_lr.score(new_x_train, y_train))
print(all_data_lr.score(new_x_test, y_test))

In [ ]:
test_categorical['weather'].value_counts()

In [ ]:
x_train.columns

In [ ]:
x_test.columns

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
new_x_test.shape

In [ ]:
new_x_train.shape

In [ ]:
from sklearn.metrics import mean_squared_error
import math

def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

In [ ]:
import numpy as np
print(rmsle(all_data_lr.predict(new_x_train),y_train))

In [ ]:
plt.figure(figsize=(12,8))
barplot = sns.barplot(list(x_train.columns), y=regressor.coef_, orient='vertical')
plt.setp(barplot.get_xticklabels(), rotation=90); plt.grid(True);

In [ ]:
from sklearn.linear_model import Ridge

alpha = [0.01, 0.1, 1, 10, 100]

for a in alpha:
    all_data_lr = Ridge(alpha = a).fit(new_x_train, y_train)
    print('alpha:', a)
    print(all_data_lr.score(new_x_train, y_train), all_data_lr.score(new_x_test, y_test))
    print()

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
sns.distplot(y_train)
plt.subplot(1, 2, 2)
sns.distplot(pd.np.log10(y_train))

In [ ]:
all_data_lr = LinearRegression().fit(new_x_train, pd.np.log10(y_train))
print(all_data_lr.score(new_x_train, pd.np.log10(y_train)))
print(all_data_lr.score(new_x_test, pd.np.log10(y_test)))

In [ ]:
print(rmsle(all_data_lr.predict(new_x_train),pd.np.log10(y_train)))

In [ ]:
alphas = [0.01, 0.1, 1, 10, 100]
for alpha in alphas:
    all_data_lr = Ridge(alpha = alpha).fit(new_x_train, pd.np.log10(y_train))
    print('alpha:', alpha)
    print(all_data_lr.score(new_x_train, pd.np.log10(y_train)), all_data_lr.score(new_x_test, pd.np.log10(y_test)))
    print()

In [ ]:
from sklearn.metrics import r2_score

for alpha in [0.01, 0.1, 1, 10, 100]:
    all_data_lr = Ridge(alpha = alpha).fit(new_x_train, pd.np.log10(y_train))
    print('alpha:', alpha)
    print(r2_score(10**all_data_lr.predict(new_x_train), y_train))
    print(r2_score(10**all_data_lr.predict(new_x_test), y_test))
    print()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)

poly_features.fit(new_x_train)

poly_x_train = poly_features.transform(new_x_train)
poly_x_test = poly_features.transform(new_x_test)

print(new_x_train.shape)
print(poly_x_train.shape)

In [ ]:
for a in [10e6, 10e7, 10e8, 10e9, 10e10]:
    print('alpha:', a)
    all_data_lr = Ridge(alpha=a).fit(poly_x_train, pd.np.log10(y_train))
    print(all_data_lr.score(poly_x_train, pd.np.log10(y_train)), all_data_lr.score(poly_x_test, pd.np.log10(y_test)))
    print()